In [ ]:
# pip install tensorflow
# pip install torchxrayvision

In [1]:
#Importing Relevant Libraries

import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model
from imutils import paths
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

## Using ResNet50

### Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory("./Train", batch_size = 20, class_mode = 'binary', target_size = (224, 224))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory("./Validation",  batch_size = 20, class_mode = 'binary', target_size = (224, 224))

In [ ]:
from tensorflow.keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D

base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
for layer in base_model.layers:
    layer.trainable = False

base_model = Sequential()
base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
base_model.add(Dense(1, activation='sigmoid'))
base_model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])
resnet_history = base_model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 10, epochs = 10)

## Using VGG16

In [2]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images("./Dataset"))
data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
    # load the image, swap color channels, and resize it to be a fixed
    # 224x224 pixels while ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)
# convert the data and labels to NumPy arrays while scaling the pixel
# intensities to the range [0, 1]
data = np.array(data) / 255.0
labels = np.array(labels)

[INFO] loading images...


In [3]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 5*1e-2
EPOCHS = 100
BS = 13

In [4]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Train/form one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
# partition the data into training and testing splits using 60% of
# the data for training and the remaining 40% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.3, stratify=labels, random_state=42)
# initialize the training data augmentation object
trainAug = ImageDataGenerator(width_shift_range = 0.2, height_shift_range = 0.2, zoom_range = 0.2, horizontal_flip = True, fill_mode="nearest")

In [5]:
from tensorflow.keras.layers import Activation, Dropout,Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D, Lambda, Input, AveragePooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential

# load the VGG16 network, ensuring the head FC layer sets are left
# off
baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.6)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras.optimizers import Adam

# compile our model
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the head of the network
print("[INFO] training head...")
H = model.fit_generator(
    trainAug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

[INFO] compiling model...
[INFO] training head...


C:\Users\Harshvardhan Agarwal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
28/28 [==============================] - 175s 6s/step - loss: 2.1507 - accuracy: 0.5817 - val_loss: 0.6592 - val_accuracy: 0.6398
Epoch 2/100
28/28 [==============================] - 196s 7s/step - loss: 0.6713 - accuracy: 0.6233 - val_loss: 0.6536 - val_accuracy: 0.6398
Epoch 3/100
28/28 [==============================] - 209s 8s/step - loss: 0.6632 - accuracy: 0.6260 - val_loss: 0.6553 - val_accuracy: 0.6398
Epoch 4/100
28/28 [==============================] - 243s 9s/step - loss: 0.6544 - accuracy: 0.6456 - val_loss: 0.6536 - val_accuracy: 0.6398
Epoch 5/100
28/28 [==============================] - 202s 7s/step - loss: 0.6527 - accuracy: 0.6427 - val_loss: 0.6537 - val_accuracy: 0.6398
Epoch 6/100
28/28 [==============================] - 167s 6s/step - loss: 0.6566 - accuracy: 0.6371 - val_loss: 0.6536 - val_accuracy: 0.6398
Epoch 7/100
28/28 [==============================] - 173s 6s/step - loss: 0.6525 - accuracy: 0.6454 - val_loss: 0.6536 - val_accuracy: 0.6398
Epoch 